In [1]:
import json
from selenium import webdriver
import re
import pandas as pd
import time
from time import sleep

In [2]:
with open('./data (1).json') as f:
    data = json.load(f)

In [3]:
data = list(filter(None, data))

In [4]:
data[0].keys()

dict_keys(['pmid', 'doi', 'title', 'abstract', 'journal', 'cited_num', 'publish_date', 'authors', 'keywords', 'cite'])

# sample data

In [5]:
sample_data = data[:10]

In [6]:
driver=webdriver.Chrome('/Users/esther/Downloads/chromedriver')
for i in sample_data:
    pmid = i['pmid']
    driver.get('http://www.ncbi.nlm.nih.gov/pubmed/'+pmid)
    
    # cited_num
    amount = driver.find_elements_by_class_name('amount')
    amount_list = [i.text for i in amount]
    if amount_list == []:
        cited_num = '0'
    elif len(amount_list) == 1:
        cited_num = str(int(amount_list[0]))
    else:
        cited_num = 'Error'
        
        
    # MLA format citation
    driver.find_element_by_css_selector(
        "#article-page > aside > div > div.actions-buttons.sidebar > div > button.citation-button.citation-dialog-trigger"
        ).click()
    time.sleep(2)
    try:
        driver.find_element_by_xpath(
                '//*[@id="article-page"]/div[1]/div/div[2]/div[2]/div/select/option[3]'
                                        ).click()
    except:
        try:
            driver.find_element_by_xpath(
                '//*[@id="article-page"]/div[2]/div/div[2]/div[2]/div/select/option[3]'
                                        ).click()
        except:
            pass
    citation_text = driver.find_elements_by_class_name('citation-text')
    cite = [i.text for i in citation_text][-1]
    
    
    # Keywords
    abstract = driver.find_elements_by_class_name('abstract')
    text = [i.text for i in abstract][-1]
    last_text = text.split('\n')[-1]
    if last_text[:8] == 'Keywords':
        keywords = re.sub('[\.]', '',last_text)
        keywords = keywords[10:].split('; ')
    else:
        keywords = []
            
            
    
    # 채워 넣기
    i['cited_num'] = cited_num
    i['cite'] = cite
    i['keywords'] = keywords
    
driver.close()

In [7]:
df_sample_data = pd.DataFrame.from_dict(sample_data)
df_sample_data

,pmid,doi,title,abstract,journal,cited_num,publish_date,authors,keywords,cite
0,6115317,10.1056/NEJM198110013051407,Case records of the Massachusetts General Hosp...,,N Engl J Med.,1,1981-10-01,[],[],“Case records of the Massachusetts General Hos...
1,6115878,,Transcutaneous nerve stimulation: its signific...,"In summation, the neuranatomy and neurophysiol...",J Foot Surg.,0,1981 Fall,"[Grosack MA, Gibbons RW, Cohen G.]",[],"Grosack, M A et al. “Transcutaneous nerve stim..."
2,6116471,10.1016/s0196,Mallory-Weiss syndrome: an unusual complicatio...,A young presented to the emergency department ...,Ann Emerg Med.,6,1981-10-04,"[Tandberg D, Liechty EJ, Fishbein D.]",[],"Tandberg, D et al. “Mallory-Weiss syndrome: an..."
3,6116552,,Correction of equinus deformity following midt...,Transtarsal amputation in a 49-year-old man wa...,Clin Orthop Relat Res.,0,1981-10-04,[Zaricznyj B.],[],"Zaricznyj, B. “Correction of equinus deformity..."
4,6117203,10.1176/ajp.138.10.1392a,Long-term lithium treatment of psychotropic-in...,,Am J Psychiatry.,0,1981-10-04,"[Yassa R, Nair V.]",[],"Yassa, R, and V Nair. “Long-term lithium treat..."
5,6117298,10.1136/hrt.46.4.446,Takayasu's disease in two brothers. Analysis o...,Recent studies indicate that certain haplotype...,Br Heart J.,4,1981-10-04,"[Makino N, Senda Y, Yamaguchi Y.]",[],"Makino, N et al. “Takayasu's disease in two br..."
6,6117386,,Recurrent acute renal failure due to nonaccide...,A 7-year-old boy experienced recurrent attacks...,Clin Nephrol.,2,1981-10-04,"[Proesmans W, Sina JK, Debucquoy P, Renoirt...",[],"Proesmans, W et al. “Recurrent acute renal fai..."
7,6117600,,Oral overdose of propylhexedrine.,,J Forensic Sci.,0,1981-10-04,"[Riddick L, Reisch R.]",[],"Riddick, L, and R Reisch. “Oral overdose of pr..."
8,6118185,,The inadvisability of neuroleptic medication i...,The use of neuroleptic drugs in the management...,Biol Psychiatry.,3,1981-10-04,[Tu J.],[],"Tu, J. “The inadvisability of neuroleptic medi..."
9,6118274,10.1007/BF00441321,Intractable diarrhoea in a patient with vasoac...,A 1 1/2 year old child developed profuse water...,Eur J Pediatr.,5,1981-10-04,"[Tiedemann K, Pritchard J, Long R, Bloom SR.]",[],"Tiedemann, K et al. “Intractable diarrhoea in ..."


### sample_data의 페이지에는 keywords가 존재 하지 않음

In [8]:
# keywords가 존재하는 data 예시
driver=webdriver.Chrome('/Users/esther/Downloads/chromedriver')
driver.get('http://www.ncbi.nlm.nih.gov/pubmed/28955658')


abstract = driver.find_elements_by_class_name('abstract')
text = [i.text for i in abstract][-1]
last_text = text.split('\n')[-1]
if last_text[:8] == 'Keywords':
    keywords = re.sub('[\.]', '',last_text)
    keywords = keywords[10:].split('; ')
else:
    keywords = []
    
driver.close()

keywords

['CASTLE',
 'case report',
 'radiation therapy',
 'thyroid carcinoma',
 'thyroid tumor']